In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import utils
import time

from keras.models import Model, Input
from keras.layers import Dense, Conv1D, BatchNormalization, GlobalMaxPooling1D, Dropout, \
    Embedding, Concatenate, SpatialDropout1D, MaxPooling1D

from utils.preprocessing_utils import tokenize_sentences, convert_tokens_to_padded_sequence, remove_punctuation, remove_punctuation_weak
from utils.dataset_utils import load_data_from_csv
from utils.embedding_utils import create_embeddings_mapping
from utils.training_utils import train_and_evaluate_model

In [12]:
import importlib
import utils
importlib.reload(utils.embedding_utils)
importlib.reload(utils.dataset_utils)
importlib.reload(utils.preprocessing_utils)
importlib.reload(utils.training_utils)
importlib.reload(utils.keras_utils)

<module 'utils.keras_utils' from '/home/philipp/work/gitprojects/toxic-comment-experiments/utils/keras_utils.py'>

Global parameters which hold for all models

In [14]:
random_seed = 2018
classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
features = 'comment_text'
np.random.seed(random_seed)
path_train_data = 'data/kaggle/train.csv'
path_test_data = 'data/kaggle/test_complete.csv'

embedding_length = 300
path_embeddings = 'data/embeddings/GoogleNews-vectors-negative300.bin.gz'

batch_size = 64
epochs = 5

In [15]:
def convert_tokens_to_padded_indices(X_train_tok, X_test_tok, word_embedding_mapping):
    max_len_train = X_train_tok.apply(lambda x: len(x)).max()
    max_len_test = X_test_tok.apply(lambda x: len(x)).max()
    # limit length to 2000, otherwise we get a MemoryError
    print("maximum comment length: {}".format(max(max_len_train, max_len_test)))
    max_comment_length = min(2000, max(max_len_train, max_len_test))
    X_train_input = convert_tokens_to_padded_sequence(X_train_tok, word_embedding_mapping, max_comment_length)
    X_test_input = convert_tokens_to_padded_sequence(X_test_tok, word_embedding_mapping, max_comment_length)
    return X_train_input, X_test_input, max_comment_length
    
def generate_model(num_tokens, max_comment_length):
    # hyperparameters
    kernel_sizes = [3, 4, 5]
    hidden_dim = 100
    num_filters = [100, 100, 100]
    dropout = 0.4
    spatial_dropout = 0.2
    
    # model
    input = Input((max_comment_length,))
    word_emb = Embedding(input_dim=num_tokens, output_dim=embedding_length, input_length=max_comment_length)(input)
    word_emb = SpatialDropout1D(spatial_dropout)(word_emb)

    conv1 = Conv1D(kernel_size=kernel_sizes[0], filters=num_filters[0], padding='same')(word_emb)
    conv1 = GlobalMaxPooling1D()(conv1)

    conv2 = Conv1D(kernel_size=kernel_sizes[1], filters=num_filters[1], padding='same')(word_emb)
    conv2 = GlobalMaxPooling1D()(conv2)

    conv3 = Conv1D(kernel_size=kernel_sizes[2], filters=num_filters[2], padding='same')(word_emb)
    conv3 = GlobalMaxPooling1D()(conv3)

    concat4 = Concatenate()([conv1, conv2, conv3])

    fc5 = Dense(hidden_dim, activation='relu')(concat4)
    fc5 = Dropout(dropout)(fc5)
    output = Dense(len(classes), activation='sigmoid')(fc5)

    model = Model(inputs=[input], outputs=[output])
    return model

Load train and test data and pretrained word2vec embeddings

In [16]:
X_train, Y_train = load_data_from_csv(path_train_data, features, classes)
X_test, Y_test = load_data_from_csv(path_test_data, features, classes)

### Baseline preprocessing
The following preprocessing techniques are applied for the baseline:
* transformation of all characters to lowercase
* tokenization using the NLTK TweetTokenizer

In [5]:
e1_X_train_tok = tokenize_sentences(X_train)
e1_X_test_tok = tokenize_sentences(X_test)

Create input matrix

In [6]:
e1_embeddings_mapping = create_embeddings_mapping(e1_X_train_tok, e1_X_test_tok, debug=True)
e1_X_train_input, e1_X_test_input, e1_max_comment_length = \
    convert_tokens_to_padded_indices(e1_X_train_tok, e1_X_test_tok, e1_embeddings_mapping)
del e1_X_train_tok
del e1_X_test_tok

In [7]:
e1_model = generate_model(len(e1_embeddings_mapping)+1, e1_max_comment_length)
e1_scores = train_and_evaluate_model(e1_model, e1_X_train_input, Y_train, (e1_X_test_input, Y_test), \
                                    epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                    random_seed, runs=5)
e1_scores_path = 'data/scores/preprocessing/e1_scores_{}'.format(time.time())
np.save(e1_scores_path, e1_scores)

RUN 1/5
Train on 159571 samples, validate on 63978 samples
Epoch 1/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0596 - acc: 0.9797

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 train: ROC-AUC - epoch: 1 - score: 0.99381
 Tox: 0.99149 - STox: 0.99233 - Obs: 0.99547 - Thr: 0.98915 - Ins: 0.99129 - IdH: 0.98754
 train: F1 Score - epoch: 1 - score: 0.78471
 Tox: 0.85237 - STox: 0.02585 - Obs: 0.84175 - Thr: 0.00000 - Ins: 0.76205 - IdH: 0.18444

 val: ROC-AUC - epoch: 1 - score: 0.97733
 Tox: 0.96147 - STox: 0.98600 - Obs: 0.97441 - Thr: 0.97437 - Ins: 0.97003 - IdH: 0.97144
 val: F1 Score - epoch: 1 - score: 0.63049
 Tox: 0.65222 - STox: 0.03675 - Obs: 0.67616 - Thr: 0.00000 - Ins: 0.63590 - IdH: 0.14948
159571/159571 [==============================] - 685s 4ms/step - loss: 0.0596 - acc: 0.9797 - val_loss: 0.0725 - val_acc: 0.9698
Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0378 - acc: 0.9849
 train: ROC-AUC - epoch: 2 - score: 0.99709
 Tox: 0.99655 - STox: 0.99446 - Obs: 0.99756 - Thr: 0.99738 - Ins: 0.99510 - IdH: 0.99557
 train: F1 Score - epoch: 2 - score: 0.85290
 Tox: 0.90255 - STox: 0.44962 - Obs: 0.89142 - Thr: 0.22262 

Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0382 - acc: 0.9848
 train: ROC-AUC - epoch: 2 - score: 0.99710
 Tox: 0.99649 - STox: 0.99440 - Obs: 0.99744 - Thr: 0.99601 - Ins: 0.99485 - IdH: 0.99570
 train: F1 Score - epoch: 2 - score: 0.82098
 Tox: 0.87566 - STox: 0.14326 - Obs: 0.87976 - Thr: 0.05285 - Ins: 0.80681 - IdH: 0.43095

 val: ROC-AUC - epoch: 2 - score: 0.97896
 Tox: 0.96121 - STox: 0.98507 - Obs: 0.97502 - Thr: 0.98280 - Ins: 0.97052 - IdH: 0.97537
 val: F1 Score - epoch: 2 - score: 0.64368
 Tox: 0.66093 - STox: 0.10959 - Obs: 0.69049 - Thr: 0.12389 - Ins: 0.64052 - IdH: 0.38696
159571/159571 [==============================] - 677s 4ms/step - loss: 0.0382 - acc: 0.9848 - val_loss: 0.0699 - val_acc: 0.9721
Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0293 - acc: 0.9881
 train: ROC-AUC - epoch: 3 - score: 0.99838
 Tox: 0.99813 - STox: 0.99590 - Obs: 0.99872 - Thr: 0.99792 - Ins: 0.99703 - IdH: 0.99806
 train: F1

Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0296 - acc: 0.9880
 train: ROC-AUC - epoch: 3 - score: 0.99833
 Tox: 0.99819 - STox: 0.99613 - Obs: 0.99864 - Thr: 0.99837 - Ins: 0.99672 - IdH: 0.99771
 train: F1 Score - epoch: 3 - score: 0.88881
 Tox: 0.93671 - STox: 0.56479 - Obs: 0.91968 - Thr: 0.50952 - Ins: 0.86613 - IdH: 0.66526

 val: ROC-AUC - epoch: 3 - score: 0.97764
 Tox: 0.95977 - STox: 0.98589 - Obs: 0.97287 - Thr: 0.98211 - Ins: 0.96946 - IdH: 0.97676
 val: F1 Score - epoch: 3 - score: 0.61541
 Tox: 0.60541 - STox: 0.32800 - Obs: 0.66140 - Thr: 0.32530 - Ins: 0.64043 - IdH: 0.54123
159571/159571 [==============================] - 681s 4ms/step - loss: 0.0296 - acc: 0.9880 - val_loss: 0.0858 - val_acc: 0.9636
Epoch 4/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0232 - acc: 0.9907
 train: ROC-AUC - epoch: 4 - score: 0.99904
 Tox: 0.99905 - STox: 0.99739 - Obs: 0.99928 - Thr: 0.99912 - Ins: 0.99833 - IdH: 0.99865
 train: F1

### Strip more than 3 of the same characters in a row
In addition to the baseline techniques all characters which occur more than 3 times in a row are stripped to a length of 3.

In [10]:
e2_X_train_tok = tokenize_sentences(X_train, reduce_len=True)
e2_X_test_tok = tokenize_sentences(X_test, reduce_len=True)

Create input matrix

In [11]:
e2_embeddings_mapping = create_embeddings_mapping(e2_X_train_tok, e2_X_test_tok, debug=True)
e2_X_train_input, e2_X_test_input, e2_max_comment_length = \
    convert_tokens_to_padded_indices(e2_X_train_tok, e2_X_test_tok, e2_embeddings_mapping)
del e2_X_train_tok
del e2_X_test_tok

maximum comment length: 3801


In [8]:
e2_model = generate_model(len(e2_embeddings_mapping)+1, e2_max_comment_length)
e2_scores = train_and_evaluate_model(e2_model, e2_X_train_input, Y_train, (e2_X_test_input, Y_test), \
                                    epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                    random_seed, runs=5)
e2_scores_path = 'data/scores/preprocessing/e2_scores_{}'.format(time.time())
np.save(e2_scores_path, e2_scores)

RUN 1/5
Train on 159571 samples, validate on 63978 samples
Epoch 1/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0599 - acc: 0.9796

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 train: ROC-AUC - epoch: 1 - score: 0.99365
 Tox: 0.99139 - STox: 0.99226 - Obs: 0.99538 - Thr: 0.98887 - Ins: 0.99139 - IdH: 0.98502
 train: F1 Score - epoch: 1 - score: 0.78571
 Tox: 0.85259 - STox: 0.08894 - Obs: 0.84118 - Thr: 0.00000 - Ins: 0.77251 - IdH: 0.01549

 val: ROC-AUC - epoch: 1 - score: 0.97748
 Tox: 0.96131 - STox: 0.98625 - Obs: 0.97495 - Thr: 0.97707 - Ins: 0.97112 - IdH: 0.96805
 val: F1 Score - epoch: 1 - score: 0.62819
 Tox: 0.64749 - STox: 0.11111 - Obs: 0.67641 - Thr: 0.00000 - Ins: 0.64308 - IdH: 0.02216
159571/159571 [==============================] - 684s 4ms/step - loss: 0.0599 - acc: 0.9796 - val_loss: 0.0742 - val_acc: 0.9691
Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0377 - acc: 0.9849
 train: ROC-AUC - epoch: 2 - score: 0.99708
 Tox: 0.99665 - STox: 0.99436 - Obs: 0.99765 - Thr: 0.99726 - Ins: 0.99499 - IdH: 0.99561
 train: F1 Score - epoch: 2 - score: 0.85625
 Tox: 0.91358 - STox: 0.42182 - Obs: 0.88444 - Thr: 0.28227 

Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0386 - acc: 0.9846
 train: ROC-AUC - epoch: 2 - score: 0.99696
 Tox: 0.99653 - STox: 0.99427 - Obs: 0.99714 - Thr: 0.99595 - Ins: 0.99449 - IdH: 0.99474
 train: F1 Score - epoch: 2 - score: 0.82349
 Tox: 0.88468 - STox: 0.14389 - Obs: 0.87916 - Thr: 0.01247 - Ins: 0.80748 - IdH: 0.33754

 val: ROC-AUC - epoch: 2 - score: 0.97837
 Tox: 0.96082 - STox: 0.98525 - Obs: 0.97445 - Thr: 0.98292 - Ins: 0.96997 - IdH: 0.97430
 val: F1 Score - epoch: 2 - score: 0.63208
 Tox: 0.64052 - STox: 0.11404 - Obs: 0.68128 - Thr: 0.00935 - Ins: 0.65048 - IdH: 0.34038
159571/159571 [==============================] - 678s 4ms/step - loss: 0.0386 - acc: 0.9846 - val_loss: 0.0728 - val_acc: 0.9699
Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0296 - acc: 0.9880
 train: ROC-AUC - epoch: 3 - score: 0.99836
 Tox: 0.99837 - STox: 0.99554 - Obs: 0.99864 - Thr: 0.99736 - Ins: 0.99678 - IdH: 0.99764
 train: F1

Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0299 - acc: 0.9879
 train: ROC-AUC - epoch: 3 - score: 0.99834
 Tox: 0.99831 - STox: 0.99567 - Obs: 0.99863 - Thr: 0.99815 - Ins: 0.99686 - IdH: 0.99759
 train: F1 Score - epoch: 3 - score: 0.88867
 Tox: 0.94145 - STox: 0.48540 - Obs: 0.91872 - Thr: 0.49619 - Ins: 0.87400 - IdH: 0.55014

 val: ROC-AUC - epoch: 3 - score: 0.97747
 Tox: 0.95926 - STox: 0.98649 - Obs: 0.97185 - Thr: 0.98537 - Ins: 0.96961 - IdH: 0.97639
 val: F1 Score - epoch: 3 - score: 0.61084
 Tox: 0.60182 - STox: 0.31201 - Obs: 0.64567 - Thr: 0.31469 - Ins: 0.64955 - IdH: 0.48231
159571/159571 [==============================] - 679s 4ms/step - loss: 0.0299 - acc: 0.9879 - val_loss: 0.0836 - val_acc: 0.9638
Epoch 4/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0233 - acc: 0.9905
 train: ROC-AUC - epoch: 4 - score: 0.99901
 Tox: 0.99911 - STox: 0.99713 - Obs: 0.99935 - Thr: 0.99876 - Ins: 0.99822 - IdH: 0.99856
 train: F1

### Remove all punctuation

In [21]:
e3_X_train_tok = tokenize_sentences(remove_punctuation(X_train))
e3_X_test_tok = tokenize_sentences(remove_punctuation(X_test))

Create input matrix

In [22]:
e3_embeddings_mapping = create_embeddings_mapping(e3_X_train_tok, e3_X_test_tok, debug=True)
e3_X_train_input, e3_X_test_input, e3_max_comment_length = \
    convert_tokens_to_padded_indices(e3_X_train_tok, e3_X_test_tok, e3_embeddings_mapping)
del e3_X_train_tok
del e3_X_test_tok

Number of unique tokens: 334853
maximum comment length: 2321


In [23]:
e3_model = generate_model(len(e3_embeddings_mapping)+1, e3_max_comment_length)
e3_scores = train_and_evaluate_model(e3_model, e3_X_train_input, Y_train, (e3_X_test_input, Y_test), \
                                    epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                    random_seed, runs=5)
e3_scores_path = 'data/scores/preprocessing/e3_scores_{}'.format(time.time())
np.save(e3_scores_path, e3_scores)

RUN 1/5


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:94: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 100456200 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 159571 samples, validate on 63978 samples
Epoch 1/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0605 - acc: 0.9793

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 train: ROC-AUC - epoch: 1 - score: 0.99355
 Tox: 0.99059 - STox: 0.99162 - Obs: 0.99556 - Thr: 0.98476 - Ins: 0.99092 - IdH: 0.98675
 train: F1 Score - epoch: 1 - score: 0.77730
 Tox: 0.81910 - STox: 0.41462 - Obs: 0.85521 - Thr: 0.00000 - Ins: 0.77217 - IdH: 0.00142

 val: ROC-AUC - epoch: 1 - score: 0.97835
 Tox: 0.96190 - STox: 0.98688 - Obs: 0.97546 - Thr: 0.97124 - Ins: 0.97059 - IdH: 0.97087
 val: F1 Score - epoch: 1 - score: 0.63993
 Tox: 0.67977 - STox: 0.41582 - Obs: 0.66263 - Thr: 0.00000 - Ins: 0.64077 - IdH: 0.00000
159571/159571 [==============================] - 691s 4ms/step - loss: 0.0605 - acc: 0.9793 - val_loss: 0.0748 - val_acc: 0.9697
Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0379 - acc: 0.9849
 train: ROC-AUC - epoch: 2 - score: 0.99720
 Tox: 0.99672 - STox: 0.99460 - Obs: 0.99756 - Thr: 0.99764 - Ins: 0.99484 - IdH: 0.99575
 train: F1 Score - epoch: 2 - score: 0.85524
 Tox: 0.91442 - STox: 0.32430 - Obs: 0.89210 - Thr: 0.18045 

Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0383 - acc: 0.9848
 train: ROC-AUC - epoch: 2 - score: 0.99729
 Tox: 0.99676 - STox: 0.99462 - Obs: 0.99771 - Thr: 0.99605 - Ins: 0.99482 - IdH: 0.99594
 train: F1 Score - epoch: 2 - score: 0.84974
 Tox: 0.90461 - STox: 0.41656 - Obs: 0.88933 - Thr: 0.09921 - Ins: 0.82410 - IdH: 0.63555

 val: ROC-AUC - epoch: 2 - score: 0.97891
 Tox: 0.96042 - STox: 0.98675 - Obs: 0.97540 - Thr: 0.98272 - Ins: 0.96891 - IdH: 0.97513
 val: F1 Score - epoch: 2 - score: 0.64073
 Tox: 0.64669 - STox: 0.28956 - Obs: 0.68010 - Thr: 0.13913 - Ins: 0.64811 - IdH: 0.54097
159571/159571 [==============================] - 687s 4ms/step - loss: 0.0383 - acc: 0.9848 - val_loss: 0.0707 - val_acc: 0.9701
Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0287 - acc: 0.9883
 train: ROC-AUC - epoch: 3 - score: 0.99841
 Tox: 0.99842 - STox: 0.99558 - Obs: 0.99865 - Thr: 0.99824 - Ins: 0.99703 - IdH: 0.99781
 train: F1

Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0292 - acc: 0.9881
 train: ROC-AUC - epoch: 3 - score: 0.99852
 Tox: 0.99863 - STox: 0.99551 - Obs: 0.99885 - Thr: 0.99842 - Ins: 0.99705 - IdH: 0.99784
 train: F1 Score - epoch: 3 - score: 0.88266
 Tox: 0.94240 - STox: 0.23244 - Obs: 0.90838 - Thr: 0.60915 - Ins: 0.87506 - IdH: 0.55321

 val: ROC-AUC - epoch: 3 - score: 0.97800
 Tox: 0.95917 - STox: 0.98604 - Obs: 0.97434 - Thr: 0.98235 - Ins: 0.97082 - IdH: 0.97423
 val: F1 Score - epoch: 3 - score: 0.64251
 Tox: 0.64539 - STox: 0.22930 - Obs: 0.68277 - Thr: 0.42135 - Ins: 0.66081 - IdH: 0.45527
159571/159571 [==============================] - 688s 4ms/step - loss: 0.0292 - acc: 0.9881 - val_loss: 0.0751 - val_acc: 0.9706
Epoch 4/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0221 - acc: 0.9910
 train: ROC-AUC - epoch: 4 - score: 0.99922
 Tox: 0.99945 - STox: 0.99715 - Obs: 0.99931 - Thr: 0.99878 - Ins: 0.99859 - IdH: 0.99895
 train: F1

### Remove all punctuation except for .,!?

In [24]:
e4_X_train_tok = tokenize_sentences(remove_punctuation_weak(X_train))
e4_X_test_tok = tokenize_sentences(remove_punctuation_weak(X_test))

Create input matrix

In [ ]:
e4_embeddings_mapping = create_embeddings_mapping(e4_X_train_tok, e4_X_test_tok, debug=True)
e4_X_train_input, e4_X_test_input, e4_max_comment_length = \
    convert_tokens_to_padded_indices(e4_X_train_tok, e4_X_test_tok, e4_embeddings_mapping)
del e4_X_train_tok
del e4_X_test_tok

Number of unique tokens: 330219
maximum comment length: 2321


In [ ]:
e4_model = generate_model(len(e4_embeddings_mapping)+1, e4_max_comment_length)
e4_scores = train_and_evaluate_model(e4_model, e4_X_train_input, Y_train, (e4_X_test_input, Y_test), \
                                    epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                    random_seed, runs=5)
e4_scores_path = 'data/scores/preprocessing/e4_scores_{}'.format(time.time())
np.save(e4_scores_path, e4_scores)

RUN 1/5
Train on 159571 samples, validate on 63978 samples
Epoch 1/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0612 - acc: 0.9792

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 train: ROC-AUC - epoch: 1 - score: 0.99342
 Tox: 0.99097 - STox: 0.99166 - Obs: 0.99521 - Thr: 0.98581 - Ins: 0.99103 - IdH: 0.98213
 train: F1 Score - epoch: 1 - score: 0.78101
 Tox: 0.84169 - STox: 0.41024 - Obs: 0.84991 - Thr: 0.00000 - Ins: 0.74243 - IdH: 0.00000

 val: ROC-AUC - epoch: 1 - score: 0.97778
 Tox: 0.96332 - STox: 0.98715 - Obs: 0.97454 - Thr: 0.97537 - Ins: 0.97141 - IdH: 0.96516
 val: F1 Score - epoch: 1 - score: 0.62966
 Tox: 0.67013 - STox: 0.40000 - Obs: 0.64058 - Thr: 0.00000 - Ins: 0.63349 - IdH: 0.00000
159571/159571 [==============================] - 684s 4ms/step - loss: 0.0612 - acc: 0.9792 - val_loss: 0.0772 - val_acc: 0.9687
Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0382 - acc: 0.9848
 train: ROC-AUC - epoch: 2 - score: 0.99707
 Tox: 0.99639 - STox: 0.99429 - Obs: 0.99779 - Thr: 0.99587 - Ins: 0.99505 - IdH: 0.99539
 train: F1 Score - epoch: 2 - score: 0.85690
 Tox: 0.91082 - STox: 0.39092 - Obs: 0.89762 - Thr: 0.19557 

Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0388 - acc: 0.9846
 train: ROC-AUC - epoch: 2 - score: 0.99668
 Tox: 0.99649 - STox: 0.99423 - Obs: 0.99733 - Thr: 0.99496 - Ins: 0.99395 - IdH: 0.99342
 train: F1 Score - epoch: 2 - score: 0.82702
 Tox: 0.87735 - STox: 0.60027 - Obs: 0.87993 - Thr: 0.13127 - Ins: 0.80713 - IdH: 0.40800

 val: ROC-AUC - epoch: 2 - score: 0.97952
 Tox: 0.96258 - STox: 0.98658 - Obs: 0.97676 - Thr: 0.98312 - Ins: 0.97078 - IdH: 0.97512
 val: F1 Score - epoch: 2 - score: 0.64436
 Tox: 0.66407 - STox: 0.37446 - Obs: 0.69537 - Thr: 0.10435 - Ins: 0.64223 - IdH: 0.39669
159571/159571 [==============================] - 680s 4ms/step - loss: 0.0388 - acc: 0.9846 - val_loss: 0.0720 - val_acc: 0.9706
Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0297 - acc: 0.9877
 train: ROC-AUC - epoch: 3 - score: 0.99836
 Tox: 0.99833 - STox: 0.99564 - Obs: 0.99874 - Thr: 0.99828 - Ins: 0.99676 - IdH: 0.99706
 train: F1